# Database Cleaning

The survey was applied in a Google Form. The data was exported to the CSV file 'Ecological Inteligence.csv'. We cleaned the data with Python because it was easier to manipulate the data than in R.

First, we download the CSV file from Google Forms: [link to the file](https://docs.google.com/forms/d/1Fsd6EXfChcF_qHH-MbgcP4FffkvTIyjmq9c837x7HQY/edit).

The following code cleans the data. We did the following steps:
1. Read the CSV file.
2. Transform the data to a pandas DataFrame.
3. Do a first cleaning of the column 'program'. It still needs to properly classify all program descriptions.
4. Clean the column 'nationality'.
5. Code all the responses of the Likert scale to 1 to 5.

In [2]:
import pandas as pd

path = './../data/'
dfenc = pd.read_csv(path + 'Ecological Inteligence Scale 2.0.csv')
dfenc.columns = [
    "date", "student", "program", "campus", "sex", "nationality", "age", "holis1", "holis2", "holis3", "holis4", "holis5", 
    "social1", "social2", "social3", "social4", "econ1", "econ2", "econ3", "change", "argument"
]
dfenc.tail(3)

,date,student,program,campus,sex,nationality,age,holis1,holis2,holis3,...,holis5,social1,social2,social3,social4,econ1,econ2,econ3,change,argument
318,2025/06/04 11:31:58 AM MDT,319,LAET,Campus Monterrey,Mujer,Mexicana,21 a 22 años,Partly agree,Not sure,Completely agree,...,Partly agree,Completely agree,Not sure,Partly agree,Partly agree,Partly agree,Partly agree,Partly agree,9,Al principio pensaba que era solo un tema de c...
319,2025/06/04 11:34:24 AM MDT,320,LAET,Campus Monterrey,Mujer,Mexicana,21 a 22 años,Completely agree,Partly agree,Completely agree,...,Completely agree,Completely agree,Completely agree,Completely agree,Partly agree,Not sure,Not sure,Partly agree,9,"Gracias a los modelos de Triple Bottom Line, C..."
320,2025/06/04 11:46:13 AM MDT,321,LAET,Campus Monterrey,Hombre,México,23 a 24 años,Completely agree,Not sure,Completely agree,...,Not sure,Not sure,Completely disagree,Partly agree,Partly agree,Partly agree,Partly agree,Partly agree,10,"Considero que es un tema recurrente, y de gran..."


Transform date to datetime.

In [3]:
from datetime import datetime

# Remove timezone info and convert to datetime
dfenc['date'] = dfenc['date'].str.replace(r' [A-Z]{2,}$', '', regex=True)
dfenc['date'] = pd.to_datetime(dfenc['date'], format='%Y/%m/%d %I:%M:%S %p')

records_per_date = dfenc['date'].dt.date.value_counts().sort_index()
records_per_date_df = records_per_date.reset_index()
records_per_date_df.columns = ['date', 'count']
records_per_date_df

,date,count
0,2025-05-06,46
1,2025-05-08,10
2,2025-05-12,20
3,2025-05-14,17
4,2025-05-15,24
5,2025-05-19,17
6,2025-05-20,53
7,2025-05-22,21
8,2025-05-23,58
9,2025-05-24,2


In [4]:
dfenc['program'] = dfenc['program'].str.strip().replace(r'\s+', ' ', regex=True)
dfenc['program'] = dfenc['program'].apply(lambda x: 'LAF' if isinstance(x, str) and x.lower() == 'laf' else x)
dfenc['program'] = dfenc['program'].apply(lambda x: 'LAET' if isinstance(x, str) and x.lower() == 'laet' else x)
program_map = {
    'LAE': 'LAET',
    'Finanzas': 'LAF',
    'Estrategia y Transformación de Negocios': 'LAET',
    'Licenciatura en Finanzas': 'LAF',
    'LAET19': 'LAET',
    'LAET con doble titulación en LIN': 'LAET',
    'LAF y LIN': 'LAF',
    'LAE19 Y Master in International Management by Eada Business School Barcelona': 'LAET',
    'Profesor': 'Ninguno',
    'LAF/ LIT': 'LAF',
    'LAET Y FINANZAS': 'LAET',
    'finanzas': 'LAF',
    'Estrategia y transformation de negocios (LAET)': 'LAET',
    'LAF 19': 'LAF',
    'LAET 19': 'LAET',
    'LAET y LAF': 'LAET',
    'Administración': 'LAET',
    'Estrategia y transformación de negocios': 'LAET',
    'LAET Y LAF': 'LAET',
    'LAET / LAF': 'LAET',
    'LAR': 'Ninguno',
    'LAF y LEM': 'LAF',
    'Licenciatura en Estrategia y Transformación de Negocios': 'LAET',
    'Estrategia y transformacion de negocios': 'LAET',
    'A01570919': 'Ninguno'
}
dfenc['program'] = dfenc['program'].replace(program_map)
dfenc['program'].value_counts().reset_index()

,program,count
0,LAET,202
1,LAF,116
2,Ninguno,3


In [5]:
# Delete 'Ninguno' from the DataFrame
dfenc = dfenc[dfenc['program'] != 'Ninguno']

In [6]:
dfenc['campus'] = dfenc['campus'].str.replace('Campus', '', regex=False).str.strip()
dfenc['campus'].value_counts().reset_index()

,campus,count
0,Guadalajara,106
1,Monterrey,52
2,San Luis Potosí,38
3,Sinaloa,30
4,Aguascalientes,26
5,Toluca,22
6,Sonora Norte,17
7,Laguna,12
8,León,8
9,Chihuahua,7


In [7]:
dfenc['sex'].value_counts().reset_index()

,sex,count
0,Mujer,163
1,Hombre,155


In [8]:
from difflib import SequenceMatcher

nationstr = dfenc['nationality']
similarity_scores = nationstr.apply(lambda x: SequenceMatcher(None, str(x).strip().lower(), 'mexicana').ratio())
similarity_table = pd.DataFrame({'nationality': nationstr, 'similarity_to_mexicana': similarity_scores})
dfenc.loc[similarity_scores >= 0.875, 'nationality'] = 'Mexicana'
dfenc.loc[dfenc['nationality'].str.strip().str.lower().isin(['mexico', 'méxico', 'mex']), 'nationality'] = 'Mexicana'
dfenc.loc[dfenc['nationality'].str.contains('Estados Unidos|Americana', case=False, na=False), 'nationality'] = 'Estadounidense'
dfenc.loc[dfenc['nationality'].str.contains('Estadounidense', case=False, na=False), 'nationality'] = 'Estadounidense'
dfenc.loc[dfenc['nationality'].str.strip().str.lower() == 'colombia', 'nationality'] = 'Colombiana'
dfenc.loc[dfenc['nationality'].str.strip().str.lower() == 'chileno', 'nationality'] = 'Chilena'
dfenc['nationality'].value_counts().reset_index()

,nationality,count
0,Mexicana,306
1,Estadounidense,5
2,Ecuatoriana,2
3,Chilena,1
4,Salvadoreña,1
5,Holandesa,1
6,Colombiana,1
7,Canadiense,1


In [9]:
nationality_to_country = {
    'Mexicana': 'México',
    'Estadounidense': 'USA',
    'Ecuatoriana': 'Ecuador',
    'Chilena': 'Chile',
    'Salvadoreña': 'El Salvador',
    'Holandesa': 'Holanda',
    'Colombiana': 'Colombia',
    'Canadiense': 'Canadá'
}

dfenc['country'] = dfenc['nationality'].map(nationality_to_country)
dfenc['country'].value_counts().reset_index()

,country,count
0,México,306
1,USA,5
2,Ecuador,2
3,Chile,1
4,El Salvador,1
5,Holanda,1
6,Colombia,1
7,Canadá,1


In [10]:
dfenc['age'].value_counts().reset_index()


,age,count
0,21 a 22 años,256
1,23 a 24 años,55
2,25 años o más,7


In [11]:
response_map = {
    'Completely agree': 5,
    'Partly agree': 4,
    'Not sure': 3,
    'Partly disagree': 2,
    'Completely disagree': 1
}
dfenc.loc[:, 'holis1':'econ3'] = dfenc.loc[:, 'holis1':'econ3'].replace(response_map)
dfenc.loc[:,'holis1':'econ3']

C:\Users\L01032102\AppData\Local\Temp\ipykernel_39172\143253931.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfenc.loc[:, 'holis1':'econ3'] = dfenc.loc[:, 'holis1':'econ3'].replace(response_map)


,holis1,holis2,holis3,holis4,holis5,social1,social2,social3,social4,econ1,econ2,econ3
0,4,5,5,5,5,5,5,5,5,5,5,5
1,5,4,5,4,5,4,2,5,5,4,5,4
2,5,2,4,5,5,5,5,3,4,4,5,4
3,4,2,5,3,4,5,3,3,4,5,5,4
4,5,4,4,5,4,5,3,4,4,4,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...
316,5,4,5,4,4,4,3,5,4,4,4,4
317,3,3,4,3,3,3,4,3,4,3,4,4
318,4,3,5,2,4,5,3,4,4,4,4,4
319,5,4,5,4,5,5,5,5,4,3,3,4


## Save databse

Save clean database to a file.

In [12]:
dfenc.to_excel(path + 'Eco_Intel2_clean.xlsx', index=False)